Text Generation with Markov Chains in Python
============================================

In [62]:
files = ['text/grimm_tales.txt']

text = ""
for f in files:
    with open(f, 'r') as f:
        text += f.read()
print(text[:500])

THE GOLDEN BIRD

A certain king had a beautiful garden, and in the garden stood a tree
which bore golden apples. These apples were always counted, and about
the time when they began to grow ripe it was found that every night one
of them was gone. The king became very angry at this, and ordered the
gardener to keep watch all night under the tree. The gardener set his
eldest son to watch; but about twelve o’clock he fell asleep, and in
the morning another of the apples was missing. Then the second


In [3]:
import numpy as np

In [4]:
len(text)

517607

Weather as a Markov Chain
-------------------------

![alt text](images/markov_weather.png "Weather")

Matrix representation (rows are current state, columns are next state):

| | Sunny | Cloudy | Rainy |
| --- | --- | --- | --- |
| **Sunny** | 0.6 | 0.1 | 0.3 |
| **Cloudy** | 0.3 | 0.3 | 0.4 |
| **Rainy** | 0.3 | 0.2 | 0.5 |


Text as a Markov Chain
----------------------

**The cat ran over the dog.**

![alt text](images/markov_text1.png "Text")

Matrix representation (rows are current state, columns are next state):

| | the | cat | ran | over | dog | . |
| --- | --- | --- | --- | --- | --- | --- |
| **the** | 0 | 0.5 | 0 | 0 | 0.5 | 0 |
| **cat** | 0 | 0 | 1 | 0 | 0 | 0 |
| **ran** | 0 | 0 | 0 | 1 | 0 | 0 |
| **over** | 1 | 0 | 0 | 0 | 0 | 0 |
| **dog** | 0 | 0 | 0 | 0 | 0 | 1 |
| **.** | 0 | 0 | 0 | 0 | 0 | 1 |



Define states as the distinct word tokens

In [61]:
import re
text = re.sub("[^A-z,.!?'\n]+", " ", text)
text = re.sub("([.,!?])", r" \1 ", text)
tokens = text.lower().split()
distinct_states =list(set(tokens))
np.shape(distinct_states)

(4700,)

In [46]:
tokens[0]

'the'

Define transition matrix

In [47]:
from scipy.sparse import csr_matrix
m = csr_matrix (
        (len(distinct_states),len(distinct_states)),
        dtype = int
)

state_index = dict(
        [(state,idx_num) for idx_num, state in enumerate(distinct_states)]
)

Count transitions and fill in transition matrix

In [48]:
for i in range (len(tokens)-1):
    row = state_index[tokens[i]]
    col = state_index[tokens[i+1]]
    m[row,col] += 1
    

/home/rhyme/.local/lib/python3.6/site-packages/scipy/sparse/_index.py:84: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_intXint(row, col, x.flat[0])


In [25]:
tokens[1]

'acertainkinghadabeautifulgarden,'

In [9]:
print(m)

  (0, 2500)	4
  (0, 9566)	3
  (1, 5701)	1
  (1, 8133)	1
  (2, 2500)	1
  (3, 1075)	1
  (3, 2322)	1
  (3, 8832)	1
  (4, 4283)	1
  (4, 5780)	1
  (4, 6264)	1
  (5, 6964)	1
  (5, 8289)	1
  (6, 3434)	2
  (6, 6084)	1
  (7, 350)	1
  (7, 445)	1
  (7, 2804)	1
  (7, 4081)	1
  (7, 4117)	1
  (7, 5876)	1
  (7, 5912)	3
  (7, 6761)	1
  (7, 7556)	3
  (8, 4599)	1
  :	:
  (9681, 8289)	1
  (9682, 2500)	1
  (9683, 2500)	1
  (9683, 6790)	1
  (9684, 6790)	1
  (9685, 1639)	1
  (9685, 7952)	1
  (9686, 6264)	1
  (9687, 3609)	1
  (9687, 6632)	1
  (9688, 3381)	1
  (9689, 3381)	1
  (9689, 6960)	1
  (9689, 8832)	1
  (9689, 8963)	1
  (9690, 2500)	1
  (9691, 2139)	1
  (9691, 5474)	1
  (9692, 3657)	1
  (9692, 8367)	1
  (9693, 2322)	1
  (9694, 2500)	1
  (9695, 6964)	1
  (9696, 4548)	1
  (9697, 6264)	1


Generate new text

In [49]:
start_state_index = np.random.randint(len(distinct_states))
state = distinct_states[start_state_index]
num_sentences = 0
output = state.capitalize()
capitalize = False

while num_sentences < 3:
    row = m[state_index[state],:]
    probabilities = row/row.sum()
    probabilities = probabilities.toarray()[0]
    
    next_state_index = np.random.choice(
        len(distinct_states),
        1,
        p = probabilities
    )
    
    next_state = distinct_states[next_state_index[0]]
    
    if next_state in ('.','!','?'):
        output += next_state + '\n\n'
        capitalize = True
        num_sentences += 1
    elif next_state == ",":
        output += next_state
    else :
        if capitalize:
            output += next_state.capitalize()
            capitalize = False
        else:
            output += " " + next_state
    
    state = next_state
    
print(output)

Coat, and beat the wind had killed two wicked and drink, and said, and he, dearest, but determined to throw you without having done, and said lina loved each had to the wild animals were going to follow it is a humming a servant could offer you should like one.

And you will beat the evening passed by them out, and was the children, and a tailor went with it we will give you free, and looked at him however little calf always followed the fox had taken up her waving locks but at the cart, to say.

So clever elsie still wept with them, for she sat upon a trance then to thy true but as he seized hansel comforted and burnt up their heads, and kissed her journey.




In [11]:
    row = m[state_index[state],:]
    print(row)
    probabilities = row/row.sum()
    print(probabilities)
    probabilities = probabilities.toarray()[0]
    print(probabilities.shape)

  (0, 105)	2
  (0, 4450)	1
  (0, 9222)	2
  (0, 9566)	1
  (0, 105)	0.3333333333333333
  (0, 4450)	0.16666666666666666
  (0, 9222)	0.3333333333333333
  (0, 9566)	0.16666666666666666
(9698,)


k-Word Markov Chain
-------------------

**The cat ran over the dog.**

![alt text](images/markov_text2.png "Text")

Matrix representation (rows are current state, columns are next state):

| | the cat | cat ran | ran over | over the | the dog | dog. |
| --- | --- | --- | --- | --- | --- | --- |
| **the cat**  | 0 | 1 | 0 | 0 | 0 | 0 |
| **cat ran**  | 0 | 0 | 1 | 0 | 0 | 0 |
| **ran over** | 0 | 0 | 0 | 1 | 0 | 0 |
| **over the** | 0 | 0 | 0 | 0 | 1 | 0 |
| **the dog**  | 0 | 0 | 0 | 0 | 0 | 1 |
| **dog.**     | 0 | 0 | 0 | 0 | 0 | 1 |



Define states as consecutive token pairs

In [63]:
k = 2
import re
text = re.sub("[^A-z,.!?'\n]+", " ", text)
text = re.sub("([.,!?])", r" \1 ", text)
tokens = text.lower().split()
states = [ tuple(tokens[i:i+k]) for i in range(len(tokens) - k +1)]
distinct_states =list(set(states))
np.shape(distinct_states)

(37560, 2)

In [65]:
states[35760]

('.', 'the')

Define and fill transition matrix

In [66]:
from scipy.sparse import csr_matrix
m = csr_matrix (
        (len(distinct_states),len(distinct_states)),
        dtype = int
)

state_index = dict(
        [(state,idx_num) for idx_num, state in enumerate(distinct_states)]
)

for i in range (len(tokens)-k):
    state = tuple(tokens[i:i+k])
    next_state = tuple(tokens[i+1:i+1+k])
    row = state_index[state]
    col = state_index[next_state]
    m[row,col] += 1

Generate new text

In [ ]:
start_state_index = np.random.randint(len(distinct_states))
state = distinct_states[start_state_index]
num_sentences = 0
output = ' '.join(state).capitalize()
capitalize = False

while num_sentences < 3:
    row = m[state_index[state],:]
    probabilities = row/row.sum()
    probabilities = probabilities.toarray()[0]
    
    next_state_index = np.random.choice(
        len(distinct_states),
        1,
        p = probabilities
    )
    
    next_state = distinct_states[next_state_index[0]]
    
    if next_state[-1] in ('.','!','?'):
        output += next_state[-1] + '\n\n'
        capitalize = True
        num_sentences += 1
    elif next_state[-1] == ",":
        output += next_state[-1]
    else :
        if capitalize:
            output += next_state[-1].capitalize()
            capitalize = False
        else:
            output += " " + next_state[-1]
    
    state = next_state
    
print(output)